In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import time
from datetime import datetime
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk

# Load pre-trained face detection model
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

if faceCascade.empty():
    print("Cascade classifier not loaded. Check XML file path.")
    exit()

# Load pre-trained emotion recognition model
emotion_model = load_model('FacialEmotionR1.h5')

# Define emotions
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Get session details
session_id = input("Enter the session ID: ")
teacher_name = input("Enter the teacher's name: ")
subject = input("Enter the subject being taught: ")

# Initialize video capture
cap = cv2.VideoCapture(0)

# Initialize list to store data
data = []

# Create main window for GUI using Tkinter
root = tk.Tk()
root.title("Student Emotion Detection Dashboard")

# Create a canvas to display video feed
canvas = tk.Canvas(root, width=800, height=600)
canvas.pack()

# Create labels to display session info
session_info = tk.Label(root, text=f"Session: {session_id} | Teacher: {teacher_name} | Subject: {subject}", font=("Helvetica", 16))
session_info.pack()

# Create a label for displaying the dominant emotion in real-time
dominant_emotion_label = tk.Label(root, text="Dominant Emotion: N/A", font=("Helvetica", 16))
dominant_emotion_label.pack()

# Function to update video feed in GUI
def update_video():
    ret, frame = cap.read()
    if not ret:
        return

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Reset the frame's emotion display text for each new frame
    display_texts = []

    for (x, y, w, h) in faces:
        # Crop face from the frame
        face_roi = gray[y:y + h, x:x + w]

        # Resize face for emotion prediction
        face_roi = cv2.resize(face_roi, (48, 48))
        face_roi = face_roi / 255.0
        face_roi = np.expand_dims(face_roi, axis=0)
        face_roi = np.expand_dims(face_roi, axis=-1)

        # Predict emotion
        predicted_emotion = emotion_model.predict(face_roi)[0]
        emotion_label = emotion_labels[np.argmax(predicted_emotion)]

        # Draw rectangle around face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Display emotion label above the rectangle
        cv2.putText(frame, emotion_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        # Append data to list with timestamp
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        student_id = f"Student_{x}_{y}"
        data.append({"Timestamp": timestamp, "Session ID": session_id, "Student ID": student_id, "Emotion": emotion_label})

        # Add emotion label to the display text list for summary display
        display_texts.append(f"ID: {student_id}, Emotion: {emotion_label}")

    # Update the canvas with the frame
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_pil = Image.fromarray(frame_rgb)
    frame_tk = ImageTk.PhotoImage(image=frame_pil)
    
    # Clear the canvas and display the new frame
    canvas.delete("all")
    canvas.create_image(0, 0, anchor=tk.NW, image=frame_tk)
    canvas.image = frame_tk  # Save a reference to the image

    # Update the dominant emotion label
    if data:
        emotion_df = pd.DataFrame(data)
        dominant_emotion = emotion_df['Emotion'].mode()[0]
        dominant_emotion_label.config(text=f"Dominant Emotion: {dominant_emotion}")

    # Schedule the next frame update
    root.after(50, update_video)

# Function to generate final reports and analysis
def generate_report():
    df = pd.DataFrame(data)

    # Save DataFrame to CSV
    file_name = f"{session_id}_emotion_data.csv"
    df.to_csv(file_name, index=False)
    print(f"Data saved successfully to {file_name}")

    if 'Student ID' not in df.columns:
        messagebox.showerror("Error", "No student data was captured during the session.")
        return

    # Generate emotional engagement report
    dominant_emotions = df.groupby('Student ID')['Emotion'].agg(lambda x: x.value_counts().idxmax())
    overall_dominant_emotion = dominant_emotions.mode()[0]

    # Classroom Engagement Analysis Report
    if overall_dominant_emotion == 'neutral':
        analysis = ("The dominant emotion was 'neutral.' This suggests the class might have been well-paced, "
                    "but could benefit from more interactive activities to increase engagement.")
    elif overall_dominant_emotion == 'happy':
        analysis = ("The dominant emotion was 'happy.' This indicates that the class was likely engaging and positive. "
                    "Continue using similar strategies to maintain this positive atmosphere.")
    elif overall_dominant_emotion == 'sad':
        analysis = ("The dominant emotion was 'sad.' This could indicate that students were disengaged or found the material challenging. "
                    "Consider reviewing the material or offering additional support.")
    elif overall_dominant_emotion == 'angry':
        analysis = ("The dominant emotion was 'angry.' This might suggest frustration or disagreement with the content or pace of the class. "
                    "Try to identify the sources of frustration and address them.")
    elif overall_dominant_emotion == 'surprise':
        analysis = ("The dominant emotion was 'surprise.' This suggests the class might have been stimulating and contained unexpected elements. "
                    "Maintain the element of surprise to keep students intrigued, but ensure it's balanced with clarity.")
    elif overall_dominant_emotion == 'fear':
        analysis = ("The dominant emotion was 'fear.' This could indicate that students were anxious or intimidated by the content. "
                    "Consider creating a more supportive and encouraging environment.")
    elif overall_dominant_emotion == 'disgust':
        analysis = ("The dominant emotion was 'disgust.' This is a rare and concerning result, potentially indicating strong negative reactions. "
                    "Evaluate the content and teaching methods to understand and mitigate this reaction.")

    # Combine the analysis with the emotional engagement report
    report_text = (f"\n--- Emotional Engagement Report ---\n\n"
                   f"Session ID: {session_id}\n"
                   f"Teacher: {teacher_name}\n"
                   f"Subject: {subject}\n\n"
                   f"The overall dominant emotion during the session was: {overall_dominant_emotion}\n\n"
                   f"{analysis}\n")
    print(report_text)

    # Plot histogram of emotions
    plt.figure(figsize=(10, 6))
    dominant_emotions.value_counts().plot(kind='bar')
    plt.title(f'Overall Emotion Distribution for {session_id} - {teacher_name}')
    plt.xlabel('Emotion')
    plt.ylabel('Number of Students')
    plt.show()

    # Provide feedback to the educator based on the overall report
    messagebox.showinfo("Session Feedback", report_text)



# Start the video update
update_video()

# Add a button to generate the report at the end of the session
report_button = tk.Button(root, text="Generate Report", command=generate_report, font=("Helvetica", 14))
report_button.pack()

# Start the Tkinter event loop
root.mainloop()

cap.release()
cv2.destroyAllWindows()